In [ ]:
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import ocf_blosc2 as Blosc2
import xarray as xr
import pandas as pd
import h5py
from tqdm import tqdm

# visualize baked entries

In [ ]:
baked2020 = np.load('../baked_nonhrv_2020_v2.npy')
baked2021 = np.load('../baked_nonhrv_2021_v2.npy')

In [ ]:
dates2020 = [datetime.strptime(str(b[0]), '%y%m%d%H%M') for b in baked2020]
dates2021 = [datetime.strptime(str(b[0]), '%y%m%d%H%M') for b in baked2021]

In [ ]:
plt.figure(figsize=(20, 10))
plt.title('Baked non-HRV data for 2020')
plt.hist(dates2020, bins=[datetime(2020, 1, 1) + i * timedelta(days=1) for i in range(366)])
# put minor ticks at the days and major ticks at the months
plt.gca().xaxis.set_minor_locator(mdates.DayLocator())
plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
plt.grid(which='major', axis='x')
plt.xlim(datetime(2020, 1, 1), datetime(2020, 12, 31))
plt.tight_layout()

In [ ]:
plt.figure(figsize=(20, 10))
plt.title('Baked non-HRV data for 2021')
plt.hist(dates2021, bins=[datetime(2021, 1, 1) + i * timedelta(days=1) for i in range(365)])
# put minor ticks at the days and major ticks at the months
plt.gca().xaxis.set_minor_locator(mdates.DayLocator())
plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
plt.grid(which='major', axis='x')
plt.xlim([datetime(2021, 1, 1), datetime(2021, 12, 31)])
plt.tight_layout()

# Bake PV, nonhrv, and weather data

In [ ]:
# get all hours for which we need to bake data (ie all hours in baked + 4 hours in each gap)
times = set()
for timeint, site in baked2020:
    time = datetime.strptime(str(timeint), '%y%m%d%H%M')
    times.add(time)
    # for i in range(1, 5):
    #     times.add(time + timedelta(hours=i))
for timeint, site in baked2021:
    time = datetime.strptime(str(timeint), '%y%m%d%H%M')
    times.add(time)
    # for i in range(1, 5):
    #     times.add(time + timedelta(hours=i))

n_times = len(times)
nonhrv_times = sorted(list(times))
n_times

In [ ]:
pv = pd.concat([
        pd.read_parquet(f"/data/climatehack/official_dataset/pv/{y}/{m}.parquet").drop("generation_wh", axis=1)
        for y in (2020, 2021)
        for m in range(1, 13)
])

nonhrv = xr.open_mfdataset(
        f"/data/climatehack/official_dataset/nonhrv/*/*.zarr.zip",
        engine="zarr",
        chunks="auto"
)

weather = xr.open_mfdataset(
        f"/data/climatehack/official_dataset/weather/*/*.zarr.zip",
        engine="zarr",
        chunks="auto"
)

In [ ]:
import h5py
from tqdm import tqdm

In [ ]:
h5file = h5py.File("data.h5", "w")
ds = h5file.create_dataset("nonhrv", shape=(n_times, 12, 293, 333, 11), dtype=np.float16, chunks=(1, 12, 32, 32, 11))

for i, time in enumerate(tqdm(times)):
    ds[i] = nonhrv['data'].sel(time=slice(time, time + timedelta(minutes=55))).to_numpy()

h5file.close()

In [ ]:
nonhrv['data'].sel(time=slice(time, time + timedelta(minutes=55)))

In [ ]:
nonhrv_times_map = {t:i for i, t in enumerate(nonhrv_times)}

In [ ]:
import pickle
with open("nonhrv_times_map.pkl", "wb") as f:
    pickle.dump(nonhrv_times_map, f)

In [ ]:
import numpy as np
a = np.zeros((10,), dtype=[('a', np.uint32), ('b', np.uint32), ('c', np.bool_, (10,))])
a

In [ ]:
a['c'].any(axis=1)

# Final bake

In [ ]:
bake_index = np.load('../bake_index.npy')
bake_index

In [ ]:
dt = np.dtype([
     ('time', np.int32),
     ('site', np.int32),
     ('nonhrv_flags', np.bool_, (11,)),
     ('weather_flags', np.bool_, (38,)),
 ], align=True)
bake_index = bake_index.astype(dt)

In [ ]:
np.save('../bake_index.npy', bake_index)

In [ ]:
%%timeit
bake_index[237855]

In [ ]:
data = h5py.File("../data.fp16.h5", "r")
data.keys()

In [ ]:
data['weather'].shape

# int8 conversion

In [ ]:
import numpy as np

nonhrv_min = np.array([0.0209, 0.1356, 0., 0., 0., 0.03937, 0.0682, 0.002346, 0.013504, 0., 0.])
nonhrv_max = np.array([1., 0.975, 0.899, 0.866, 1., 0.9917, 1., 1., 1., 1., 0.7856])
weather_min = np.array([
  2.590e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
  0.000e+00,  0.000e+00,  0.000e+00, -1.032e+01, -4.028e+01, -3.334e+01,
 -1.677e+01,     np.inf,  3.576e+00, -2.157e-01, -1.979e-05,  2.489e+02,
  2.284e+02,  2.505e+02,  2.541e+02,  2.358e+02,  2.335e+02,  0.000e+00,
 -2.722e+01, -2.866e+01, -4.912e+01, -4.631e+01, -3.747e+01, -2.695e+01,
 -5.691e+01, -5.909e+01, -3.709e+01, -3.981e+01,  5.722e-02,  0.000e+00,
  0.000e+00,  2.950e-05])
weather_max = np.array([
 8.5000e+01, 5.4450e+02, 8.6400e+02, 2.2440e+03, 1.0000e+02, 1.0000e+02,
 1.0000e+02, 1.0000e+02, 4.3086e+00, 1.3172e+01, 2.6891e+01, 2.2797e+01,
 1.4117e+01,     np.inf, 1.0000e+02, 1.5375e+02, 1.2394e+02, 3.1375e+02,
 2.6975e+02, 2.9975e+02, 3.0950e+02, 3.2250e+02, 2.9775e+02, 1.5075e+02,
 2.9094e+01, 7.3375e+01, 7.9500e+01, 4.7188e+01, 3.9812e+01, 2.8469e+01,
 5.1812e+01, 6.0844e+01, 4.8625e+01, 4.2469e+01, 6.5062e+01, 1.4220e+03,
 9.6000e+01, 1.0000e+00])

nonhrv_stats = np.stack([nonhrv_min, nonhrv_max], axis=1)
weather_stats = np.stack([weather_min, weather_max], axis=1)

In [ ]:
WEATHER_KEYS = ["alb_rad", "aswdifd_s", "aswdir_s", "cape_con", "clch", "clcl", "clcm", "clct",
"h_snow", "omega_1000", "omega_700", "omega_850", "omega_950", "pmsl", "relhum_2m", "runoff_g",
"runoff_s", "t_2m", "t_500", "t_850", "t_950", "t_g", "td_2m", "tot_prec", "u_10m", "u_50", "u_500",
"u_850", "u_950", "v_10m", "v_50", "v_500", "v_850", "v_950", "vmax_10m", "w_snow", "ww", "z0"]
weather_key_map = {k:i for i, k in enumerate(WEATHER_KEYS)}

In [ ]:
weather_ranges = {}
for i, k in enumerate(WEATHER_KEYS):
    weather_ranges[k] = (weather_min[i], weather_max[i])
weather_ranges['alb_rad'] = (0, 100)
weather_ranges['ww'] = (0, 100)
weather_ranges['pmsl'] = (93928.17, 105314.26)
weather_ranges['relhum_2m'] = (0, 100)

for k, v in weather_ranges.items():
    print(f'{k:10s}: {i:4d} {v[0]:10.3f} {v[1]:10.3f}')

In [ ]:
weather_ranges.__repr__()

In [ ]:
plt.imshow(data['weather'][3, :, :, weather_key_map['pmsl']])
plt.colorbar()

# transposing

In [ ]:
WEATHER_RANGES = {'alb_rad': (0, 100), 'aswdifd_s': (0.0, 544.5), 'aswdir_s': (0.0, 864.0), 'cape_con': (0.0, 2244.0), 'clch': (0.0, 100.0), 'clcl': (0.0, 100.0), 'clcm': (0.0, 100.0), 'clct': (0.0, 100.0), 'h_snow': (0.0, 4.3086), 'omega_1000': (-10.32, 13.172), 'omega_700': (-40.28, 26.891), 'omega_850': (-33.34, 22.797), 'omega_950': (-16.77, 14.117), 'pmsl': (93928.17, 105314.26), 'relhum_2m': (0, 100), 'runoff_g': (-0.2157, 153.75), 'runoff_s': (-1.979e-05, 123.94), 't_2m': (248.9, 313.75), 't_500': (228.4, 269.75), 't_850': (250.5, 299.75), 't_950': (254.1, 309.5), 't_g': (235.8, 322.5), 'td_2m': (233.5, 297.75), 'tot_prec': (0.0, 150.75), 'u_10m': (-27.22, 29.094), 'u_50': (- 28.66, 73.375), 'u_500': (-49.12, 79.5), 'u_850': (-46.31, 47.188), 'u_950': (-37.47, 39.812), 'v_10m': (-26.95, 28.469), 'v_50': (-56.91, 51.812), 'v_500': (-59.09, 60.844), 'v_850': (-37.09, 48.625), 'v_950': (-39.81, 42.469), 'vmax_10m': (0.05722, 65.062), 'w_snow': (0.0, 1422.0), 'ww': (0, 100), 'z0': (2.95e-05, 1.0)}

WEATHER_KEYS = ["alb_rad", "aswdifd_s", "aswdir_s", "cape_con", "clch", "clcl", "clcm", "clct", "h_snow", "omega_1000", "omega_700", "omega_850", "omega_950", "pmsl", "relhum_2m", "runoff_g", "runoff_s", "t_2m", "t_500", "t_850", "t_950", "t_g", "td_2m", "tot_prec", "u_10m", "u_50", "u_500", "u_850", "u_950", "v_10m", "v_50", "v_500", "v_850", "v_950", "vmax_10m", "w_snow", "ww", "z0"]

NONHRV_KEYS = ['IR_016', 'IR_039', 'IR_087', 'IR_097', 'IR_108', 'IR_120', 'IR_134', 'VIS006', 'VIS008', 'WV_062', 'WV_073']

In [ ]:
bake_index = np.load('../bake_index.npy')
orig = h5py.File("../data.uint8.h5", "r")

In [ ]:
bake_index_dt = np.dtype([
    ('time', np.int32),
    ('site', np.int32),
    ('nonhrv_flags', np.bool_, (11,)),
    ('weather_flags', np.bool_, (38,)),
], align=True)

h5file = h5py.File("data.h5", "w")

# based on bake index, save all pv data to a single hdf5 file
ds = h5file.create_dataset(
        'bake_index',
        shape=(len(bake_index),),
        dtype=bake_index_dt,
        chunks=(min(10000, len(bake_index)),),
)
ds[:] = bake_index

ds = h5file.create_dataset(
        "nonhrv",
        shape=(11, orig['nonhrv'].shape[0], 12, 293, 333),
        dtype=np.uint8,
        chunks=(1, 16, 12, 293, 333),
)
ds.attrs['times'] = orig['nonhrv'].attrs['times']
ds.attrs['ranges'] = np.vstack((np.zeros(11), np.ones(11)), dtype=np.float32).T

print('Writing nonhrv data to hdf5 file')
for i, v in enumerate(tqdm(orig['nonhrv'])):
    ds[:, i] = v.transpose(3, 0, 1, 2)

# based on bake index, save all weather data to a single hdf5 file

ds = h5file.create_dataset(
        "weather",
        shape=(38, orig['weather'].shape[0], 305, 289),
        dtype=np.uint8,
        chunks=(1, 128, 305, 289),
)
ds.attrs['times'] = orig['weather'].attrs['times']
ds.attrs['ranges'] = np.array(list(WEATHER_RANGES.values()), dtype=np.float32)

print('Writing weather data to hdf5 file')
for i, v in enumerate(tqdm(orig['weather'])):
    ds[:, i] = v.transpose(2, 0, 1)

h5file.close()

In [ ]:
(
    (bake_index['time'] < 1609459200) & 
    bake_index['nonhrv_flags'].all(axis=1) &
    bake_index['weather_flags'].all(axis=1)
).sum()

2 experiments, loading first half of data timewise (important lol):
all channels, using some
```
old order (16 workers): 54s + 1.2-1.3s/iter
new order (16 workers): 2m at first, 50s after + 1.15s/iter
```

2 nonhrv channels (using 1) and 9 or so weather channels:
```
old order (16 workers): i didn't wanna wait
new order (16 workers): 25s + 1.15s/iter
```